In [1]:
import json
import pandas as pd
import numpy as np
import pyarrow
import urllib3
from tqdm import trange
import os
from ping3 import ping, verbose_ping


In [2]:
metadata = pd.read_parquet("train.parquet")

print(metadata.shape)

(540, 4)


In [3]:
metadata.columns

Index(['image_url', 'text', 'number', 'image'], dtype='object')

In [4]:
metadata = metadata[0:100]

In [5]:
from icmplib import ping
from icmplib.exceptions import NameLookupError


def host_up(hostname: str):
    try:
        host = ping(hostname, count=5, interval=0.2, timeout=1, privileged=False)
    except NameLookupError:
        return False
    return host.packets_sent == host.packets_received

In [6]:
import requests

def is_url_image(image_url):
   image_formats = ("image/png", "image/jpeg", "image/jpg")
   try:
      r = requests.head(image_url)
   except:
      return False
   try:
      if r.headers["content-type"] in image_formats:
         return True
   except:
         return True
      
   return False

In [7]:
metadata.iloc[0]["image_url"]

'http://www.wirerealm.com/wp-content/uploads/2015/08/top-10-best-gaming-headsets-1024x448.jpg'

In [8]:
from concurrent.futures import as_completed
from requests_futures.sessions import FuturesSession

In [11]:
## run docker img with foll command (docker run --rm -it -p 8080:8080 imadelh/opencv_tf:full) and then run 
urls = list(metadata["image_url"])
caps = list(metadata["text"])
test = "https://cdn.xxl.thumbs.canstockphoto.com/canstock13378645.jpg"
print(urls[0:5])
print(caps[0:5])

http = urllib3.PoolManager(num_pools=8)
# print(requests.head("https://hds"))
# resp = http.request("GET", f"http://0.0.0.0:8080/predict/v1?model=yolo&image_url="+str("https://cdn.xxl.thumbs.canstockphoto.com/canstock13378645.jpg"))
# # print(json.loads(str(resp.data.decode('utf-8'))))
# x = json.loads(str(resp.data.decode('utf-8')))
# print(x)
# print(len(json.loads(str(x))))
l = list()


for i in trange(100):
   
    if is_url_image(urls[i]):
        try:
            resp = http.request("GET", f"http://0.0.0.0:8080/predict/v1?model=yolo&image_url="+str(urls[i]))
        except:
            resp = 'b[]\n'
        # print(json.loads(str(resp.data.decode('utf-8'))))
        try:
            x = json.loads(str(resp.data.decode('utf-8')))
        except:
            x = []
        l.append((x,urls[i],caps[i]))
    else:
        x = []
        l.append((x,urls[i],caps[i]))


['http://www.wirerealm.com/wp-content/uploads/2015/08/top-10-best-gaming-headsets-1024x448.jpg', 'https://wmich.edu/sites/default/files/styles/720p_anamorphic/public/images/u1297/2019/lightbulb3_0.jpg?itok=9fGHQTIR&timestamp=1565960849', 'http://img0.etsystatic.com/000/0/5304297/il_fullxfull.191730668.jpg', 'https://cdn.vectorstock.com/i/thumb-large/74/37/4657437.jpg', 'https://laurelleaffarm.com/item-photos/vintage-silver-plate-tablespoons-serving-spoon-set-of-six-1847-Rogers-Ambassador-pattern-Laurel-Leaf-Farm-item-no-pw515216-1.jpg']
['We review the ten best gaming headsets in the market', 'background photo of three light bulbs', 'City prints: Set of three big prints - $150.00 USD', 'Set of eight arrows in all directions vector', 'vintage silver plate tablespoons, serving spoon set of six 1847 Rogers pattern']


100%|██████████| 100/100 [03:22<00:00,  2.02s/it]


In [3]:
# rewrite the above code cell using parallelization and sessions

valid_urls = [i for i in range(100) if is_url_image(urls[i])]
with FuturesSession() as session:
    futures = [session.get(f"http://0.0.0.0:8080/predict/v1?model=yolo&image_url=" + str(urls[el])) for el in valid_urls]
    for future in as_completed(futures):
        response = future.result()
        print(response.content.decode('utf-8'))
        answers[valid_urls[futures.index(future)]] = json.loads(response.content.decode('utf-8'))

with FuturesSession() as session:
    futures = [session.get("https://www.example.com") for _ in range(10)]
    for future in as_completed(futures):
        response = future.result()
        print(response.content)

b'<!doctype html>\n<html>\n<head>\n    <title>Example Domain</title>\n\n    <meta charset="utf-8" />\n    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />\n    <meta name="viewport" content="width=device-width, initial-scale=1" />\n    <style type="text/css">\n    body {\n        background-color: #f0f0f2;\n        margin: 0;\n        padding: 0;\n        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;\n        \n    }\n    div {\n        width: 600px;\n        margin: 5em auto;\n        padding: 2em;\n        background-color: #fdfdff;\n        border-radius: 0.5em;\n        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);\n    }\n    a:link, a:visited {\n        color: #38488f;\n        text-decoration: none;\n    }\n    @media (max-width: 700px) {\n        div {\n            margin: 0 auto;\n            width: auto;\n        }\n    }\n    </style>    \n</head>\n\n<body>\n<div>\n    

In [10]:
print(l)
for i in l:
    print(len(i))

[([{'bbox': [834.0, 238.0, 116, 200], 'confidence': 0.7781149744987488, 'label': 'scissors'}, {'bbox': [622.5, 236.5, 135, 199], 'confidence': 0.5551069378852844, 'label': 'scissors'}], 'http://www.wirerealm.com/wp-content/uploads/2015/08/top-10-best-gaming-headsets-1024x448.jpg', 'We review the ten best gaming headsets in the market'), ([], 'https://wmich.edu/sites/default/files/styles/720p_anamorphic/public/images/u1297/2019/lightbulb3_0.jpg?itok=9fGHQTIR&timestamp=1565960849', 'background photo of three light bulbs'), ([], 'http://img0.etsystatic.com/000/0/5304297/il_fullxfull.191730668.jpg', 'City prints: Set of three big prints - $150.00 USD'), ([], 'https://cdn.vectorstock.com/i/thumb-large/74/37/4657437.jpg', 'Set of eight arrows in all directions vector'), ([{'bbox': [483.5, 42.5, 129, 357], 'confidence': 0.9954971075057983, 'label': 'spoon'}, {'bbox': [28.0, 22.0, 68, 334], 'confidence': 0.9940283298492432, 'label': 'spoon'}, {'bbox': [313.5, 85.5, 77, 273], 'confidence': 0.98

In [ ]:
f = open("segments0.npy","wb")
np.save(f,pd.Series(l))

In [ ]:
len(l)


80

In [ ]:
f = open("segments0.npy","rb")

out = np.load(f,allow_pickle=True)
len(out)

80